In [1]:
from src.controller.HwpController import HwpController

import re, uuid
import numpy as np

In [2]:
controller = HwpController()
path = '/Users/dobby/Desktop/Workspace/3월 계측보고서(인덕원~동탄1공구 S01).hwp'

controller.get_table_list(path, '일자별 계측 현황')
table_list = controller.get_list()

table_list

undefined UnderlineStyle value: 15
defined name/values: {'SOLID': UnderlineStyle.SOLID, 'DASHED': UnderlineStyle.DASHED, 'DOTTED': UnderlineStyle.DOTTED, 'DASH_DOT': UnderlineStyle.DASH_DOT, 'DASH_DOT_DOT': UnderlineStyle.DASH_DOT_DOT, 'LONG_DASHED': UnderlineStyle.LONG_DASHED, 'LARGE_DOTTED': UnderlineStyle.LARGE_DOTTED, 'DOUBLE': UnderlineStyle.DOUBLE, 'LOWER_WEIGHTED': UnderlineStyle.LOWER_WEIGHTED, 'UPPER_WEIGHTED': UnderlineStyle.UPPER_WEIGHTED, 'MIDDLE_WEIGHTED': UnderlineStyle.MIDDLE_WEIGHTED}
undefined UnderlineStyle value: 15
defined name/values: {'SOLID': UnderlineStyle.SOLID, 'DASHED': UnderlineStyle.DASHED, 'DOTTED': UnderlineStyle.DOTTED, 'DASH_DOT': UnderlineStyle.DASH_DOT, 'DASH_DOT_DOT': UnderlineStyle.DASH_DOT_DOT, 'LONG_DASHED': UnderlineStyle.LONG_DASHED, 'LARGE_DOTTED': UnderlineStyle.LARGE_DOTTED, 'DOUBLE': UnderlineStyle.DOUBLE, 'LOWER_WEIGHTED': UnderlineStyle.LOWER_WEIGHTED, 'UPPER_WEIGHTED': UnderlineStyle.UPPER_WEIGHTED, 'MIDDLE_WEIGHTED': UnderlineStyle.MIDDL

[[{'row': '0', 'col': '0', 'rowspan': '2', 'text': '일자'},
  {'row': '0', 'col': '1', 'rowspan': '2', 'text': '발파횟수'},
  {'row': '0', 'col': '2', 'rowspan': '2', 'text': '발파시간'},
  {'row': '0', 'col': '3', 'rowspan': '2', 'text': '계측위치'},
  {'row': '1', 'col': '4', 'rowspan': '1', 'text': '진동속도(cm/s)'},
  {'row': '1', 'col': '5', 'rowspan': '1', 'text': '진동레벨(dB(V))'},
  {'row': '1', 'col': '6', 'rowspan': '1', 'text': '소음레벨(dB(A))'},
  {'row': '1', 'col': '7', 'rowspan': '1', 'text': '진동속도(cm/s)'},
  {'row': '1', 'col': '8', 'rowspan': '1', 'text': '진동레벨(dB(V))'},
  {'row': '1', 'col': '9', 'rowspan': '1', 'text': '소음레벨(dB(A))'},
  {'row': '2', 'col': '0', 'rowspan': '3', 'text': '3월2일'},
  {'row': '2', 'col': '1', 'rowspan': '3', 'text': '1'},
  {'row': '2', 'col': '2', 'rowspan': '3', 'text': '7:00'},
  {'row': '2', 'col': '3', 'rowspan': '1', 'text': '과천지식정보타운'},
  {'row': '2', 'col': '4', 'rowspan': '1', 'text': '0.2'},
  {'row': '2', 'col': '5', 'rowspan': '1', 'text': '75'},
  {'

In [21]:
title = ['진동속도(cm/s)', '진동레벨(dB(V))', '소음레벨(dB(A))']

columns = [ [ item['text'] for item in items if item['row'] == '0' or item['row'] == '1' ] for items in table_list ]
columns = set(tuple(items) for items in columns)
columns = [ item for items in columns for item in items ]

columns

['일자',
 '발파횟수',
 '발파시간',
 '계측위치',
 '진동속도(cm/s)',
 '진동레벨(dB(V))',
 '소음레벨(dB(A))',
 '진동속도(cm/s)',
 '진동레벨(dB(V))',
 '소음레벨(dB(A))']

In [6]:
classification_list = []
current_list = []

for items in table_list:
    for item in items[len(columns):]:
        if current_list and item['col'] =='0':
            classification_list.append(current_list)
            current_list = []
        
        current_list.append(item)
    
    if current_list:
        classification_list.append(current_list)
            
classification_list

[[{'row': '2', 'col': '0', 'rowspan': '3', 'text': '3월2일'},
  {'row': '2', 'col': '1', 'rowspan': '3', 'text': '1'},
  {'row': '2', 'col': '2', 'rowspan': '3', 'text': '7:00'},
  {'row': '2', 'col': '3', 'rowspan': '1', 'text': '과천지식정보타운'},
  {'row': '2', 'col': '4', 'rowspan': '1', 'text': '0.2'},
  {'row': '2', 'col': '5', 'rowspan': '1', 'text': '75'},
  {'row': '2', 'col': '6', 'rowspan': '1', 'text': '75'},
  {'row': '2', 'col': '7', 'rowspan': '1', 'text': '0.039'},
  {'row': '2', 'col': '8', 'rowspan': '1', 'text': '-'},
  {'row': '2', 'col': '9', 'rowspan': '1', 'text': '59.02'},
  {'row': '3', 'col': '3', 'rowspan': '1', 'text': '군포가압장'},
  {'row': '3', 'col': '4', 'rowspan': '1', 'text': '0.2'},
  {'row': '3', 'col': '5', 'rowspan': '1', 'text': '-'},
  {'row': '3', 'col': '6', 'rowspan': '1', 'text': '-'},
  {'row': '3', 'col': '7', 'rowspan': '1', 'text': '0.064'},
  {'row': '3', 'col': '8', 'rowspan': '1', 'text': '-'},
  {'row': '3', 'col': '9', 'rowspan': '1', 'text': '6

In [65]:
for items in classification_list:
    date_data = items[0]
    head_data = [ item for item in items if int(item['rowspan']) > 1 and int(item['col']) != 0 ]

    row_data = list(set([ int(row['row']) for row in items ]))
    
    for idx in row_data:
        temp_list = [ item for item in items if int(item['row']) == idx ]
        
        if not date_data in temp_list and any(item in temp_list for item in head_data):
            temp_list.insert(0, head_data)      

        print(temp_list)                          

    print()

[{'row': '2', 'col': '0', 'rowspan': '3', 'text': '3월2일'}, {'row': '2', 'col': '1', 'rowspan': '3', 'text': '1'}, {'row': '2', 'col': '2', 'rowspan': '3', 'text': '7:00'}, {'row': '2', 'col': '3', 'rowspan': '1', 'text': '과천지식정보타운'}, {'row': '2', 'col': '4', 'rowspan': '1', 'text': '0.2'}, {'row': '2', 'col': '5', 'rowspan': '1', 'text': '75'}, {'row': '2', 'col': '6', 'rowspan': '1', 'text': '75'}, {'row': '2', 'col': '7', 'rowspan': '1', 'text': '0.039'}, {'row': '2', 'col': '8', 'rowspan': '1', 'text': '-'}, {'row': '2', 'col': '9', 'rowspan': '1', 'text': '59.02'}]
[{'row': '3', 'col': '3', 'rowspan': '1', 'text': '군포가압장'}, {'row': '3', 'col': '4', 'rowspan': '1', 'text': '0.2'}, {'row': '3', 'col': '5', 'rowspan': '1', 'text': '-'}, {'row': '3', 'col': '6', 'rowspan': '1', 'text': '-'}, {'row': '3', 'col': '7', 'rowspan': '1', 'text': '0.064'}, {'row': '3', 'col': '8', 'rowspan': '1', 'text': '-'}, {'row': '3', 'col': '9', 'rowspan': '1', 'text': '60.61'}]
[{'row': '4', 'col': '3'